In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()

In [4]:
planner = Agent(
    role="Linkedin Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=True,
	verbose=True
)

writer = Agent(
    role="Linkedin Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements."
              "You need to add Hastags to the context to reach higher audience.",
    
    tools=[search_tool],
    allow_delegation=True,
    verbose=True
)

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

In [5]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [6]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
def image_description(query: str) -> str:
    qa_system_prompt = "Based on the context provided, generate a vivid and attractive description for the following image and make the description in 1 sentence:\n"
    llm = ChatOpenAI(temperature=0)
    qa_prompt = ChatPromptTemplate.from_messages([
                ("system", qa_system_prompt),
                ("human", "Context :{context}\n\nDescription :"),
            ])


    rag_chain = (qa_prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain.invoke({"context": query})

In [8]:
from openai import OpenAI
def generate_image(query: str) -> str:
    """
    Generates an image based on the given query using an AI image generation API.
    Returns the URL of the generated image.
    """
    # Image generation code here (using DALL-E, Stable Diffusion, etc.)

    client = OpenAI()

    response = client.images.generate(
    model="dall-e-3",
    prompt=query,
    size="1024x1024",
    quality="standard",
    n=1,
    )

    image_url = response.data[0].url
    return image_url

In [9]:
import requests
import os
def get_slack_post(query: str):
    # Data payload for the webhook
    # Replace with your webhook URL
    RESULT= crew.kickoff(inputs={"topic": query})
    print(RESULT)
    WEBHOOK_URL = os.environ.get("SLACK_WEBHOOK_URL")
    image_url= generate_image(image_description(RESULT))
    # Message in Markdown format
    blocks = [
        {
            "type": "header",
            "text": {
                "type": "plain_text",
                "text": "Query : " + query
            }
        },
        {
            "type": "image",
            "image_url": image_url,
            "alt_text": "Image of " + query
        }
    ]
    
    for i in RESULT.split('\n'):
        if i.strip():
            blocks.append({
                "type": "context",
                "elements": [
                    {
                        "type": "mrkdwn",
                        "text": i
                    }
                ]
            })
    
    payload = {
        "attachments": [
            {
                "blocks": blocks
            }
        ]
    }
    
    print(payload)

    # Sending the POST request to the webhook
    response = requests.post(WEBHOOK_URL, json=payload)

    # Check if the request was successful
    if response.status_code == 200:
        print("Message sent successfully!")
    else:
        print(f"Failed to send message. Status code: {response.status_code}")
        print(response.text)

In [10]:
get_slack_post("Large Language Models")

 [2024-06-25 18:55:27][DEBUG]: == Working Agent: Linkedin Content Planner
 [2024-06-25 18:55:27][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Large Language Models.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: Before I can start developing a detailed content outline, I need to understand the latest trends, key players, and noteworthy news in the field of Large Language Models. This will help me ensure the article is timely and relevant. I should delegate this task to the Linkedin Content Writer who can research and compile this information.

Action: Delegate work to coworker

Action Input: {
  "task": "Research and compile the latest trends, key players, and noteworthy news on Large Language Models. Thi

In [11]:
get_slack_post("RAPTOR Chunking in RAG Applications")

 [2024-06-25 19:00:47][DEBUG]: == Working Agent: Linkedin Content Planner
 [2024-06-25 19:00:47][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on RAPTOR Chunking in RAG Applications.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: Before developing a detailed content plan, I need to understand the latest trends, key players, and noteworthy news related to RAPTOR Chunking in RAG Applications. This will help in identifying relevant SEO keywords and crafting an engaging outline.

Action: Ask question to coworker
Action Input: {"question": "Can you provide the latest trends, key players, and any noteworthy news related to RAPTOR Chunking in RAG Applications?", "coworker": "Linkedin Content Writer", "context": "We 

In [12]:
get_slack_post("Langchain Vs CrewAI")

 [2024-06-25 19:04:33][DEBUG]: == Working Agent: Linkedin Content Planner
 [2024-06-25 19:04:33][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Langchain Vs CrewAI.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: To create a comprehensive content plan, I first need to understand the current landscape around Langchain and CrewAI, including their latest developments and how they are positioned in the market. This will help in creating a more informed and relevant content plan.

Action: Ask question to coworker
Action Input: {
  "question": "Can you provide the latest trends, key players, and noteworthy news about Langchain and CrewAI?",
  "context": "We are working on a blog article about Langchain vs CrewAI a

In [13]:
get_slack_post("Donald Trump")

 [2024-06-25 19:10:19][DEBUG]: == Working Agent: Linkedin Content Planner
 [2024-06-25 19:10:19][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Donald Trump.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: To begin creating a comprehensive content plan about Donald Trump, I need to first gather the latest trends and notable information about him. This will help in identifying the target audience and structuring the content outline effectively.

Action: Ask question to coworker

Action Input: {"question": "What are the latest trends and notable news about Donald Trump?", "coworker": "Linkedin Content Writer"}


> Entering new CrewAgentExecutor chain...
I need to find the latest trends and notable news about D